In [8]:
import numpy as np
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.data import nist
import qml

angstrom = 1 / 0.52917721067

In [9]:
22/52*6

2.5384615384615383

In [15]:
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')



mol = pyscf.gto.Mole()
for ch, coords in zip(com.nuclear_charges, com.coordinates):
    mol.atom.append([ch, coords])

mol.basis = 'def2-tzvp'
# print(total_elec - num_elec)
mol.verbose = 4
mol.nelectron = 2
mol.build()
mrhf = pyscf.scf.RHF(mol)

System: uname_result(system='Linux', node='pc-avl24', release='4.15.0-108-generic', version='#109-Ubuntu SMP Fri Jun 19 11:33:10 UTC 2020', machine='x86_64', processor='x86_64')  Threads 8
Python 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
numpy 1.16.4  scipy 1.2.1
Date: Mon Jul  6 23:24:02 2020
PySCF version 1.7.1
PySCF path  /home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 12
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 C     -0.136883959200   1.448579793000   0.011368940800 AA   -0.258673193734   2.737419078349   0.021484184438 Bohr
[INPUT]  2 C      0.064357729200  -0.043732595200   0.001817971000 AA    0.121618482187  -0.082642627644   0.003435467292 Bohr
[INPUT]  3 O      1.114424575900  -0.629340888600  -0.012303871000 AA    2.105957234936  -1.189281918444  -0.

In [20]:
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
total_elec = com.nuclear_charges.sum()
num_elecs = [2]#[22, 30, 38, 44, 52]
alchemical_potentials = []
e_homo = []
e_lumo = []

for num_elec in num_elecs:
    print(f'Calculating {num_elec}')
    lam_val = num_elec/total_elec
    # core_elec = com.nuclear_charges.sum() - total_num_ve

    mol = pyscf.gto.Mole()
    for ch, coords in zip(com.nuclear_charges, com.coordinates):
        mol.atom.append([ch, coords])

    mol.basis = 'def2-tzvp'
    print(total_elec - num_elec)
    mol.charge = total_elec - num_elec
#     mol.nelectron = num_elec
    mol.verbose = 4
    mol.build()

    method = "HF"
    if method not in ["CCSD", "HF"]:
        raise NotImplementedError("Method %s not supported." % method)

    deltaZ = -com.nuclear_charges*(1-lam_val)
    includeonly = np.arange(len(mol.atom_coords()))


    def add_qmmm(calc, mol, deltaZ):
        mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

        def energy_nuc(self):
            q = mol.atom_charges().astype(np.float)
            q[includeonly] += deltaZ
            return mol.energy_nuc(q)

        mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

        return mf

    if method == "HF":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        dm1_ao = calc.make_rdm1()
        total_energy = calc.e_tot
    if method == "CCSD":
        calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
        hfe = calc.kernel(verbose=0)
        mycc = pyscf.cc.CCSD(calc).run()
        dm1 = mycc.make_rdm1()
        dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
        total_energy = mycc.e_tot

    # GRIDLESS, as things should be ############################
    # Total energy of SCF run

    print("TOTAL_ENERGY", total_energy)

    # Electronic EPN from electron density
    alch_pot_tmp = []
    for site in includeonly:
        mol.set_rinv_orig_(mol.atom_coords()[site])
#         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
        alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
    alchemical_potentials.append(alch_pot_tmp)
    
    # energy of HOMO and LUMO
    nocc = int((total_elec - mol.charge)/2)
    e_homo.append(calc.mo_energy[nocc-1])
    e_lumo.append(calc.mo_energy[nocc])

Calculating 2
50
System: uname_result(system='Linux', node='pc-avl24', release='4.15.0-108-generic', version='#109-Ubuntu SMP Fri Jun 19 11:33:10 UTC 2020', machine='x86_64', processor='x86_64')  Threads 8
Python 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
numpy 1.16.4  scipy 1.2.1
Date: Mon Jul  6 23:30:49 2020
PySCF version 1.7.1
PySCF path  /home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 12
[INPUT] num. electrons = 2
[INPUT] charge = 50
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 C     -0.136883959200   1.448579793000   0.011368940800 AA   -0.258673193734   2.737419078349   0.021484184438 Bohr
[INPUT]  2 C      0.064357729200  -0.043732595200   0.001817971000 AA    0.121618482187  -0.082642627644   0.003435467292 Bohr
[INPUT]  3 O      1.114424575900  -0.629340888600  -0.012303871000 AA    2.105957234936  -1

/home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf/gto/mole.py:1047: UserWarning: Function mol.dumps drops attribute charge because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= 0.740642695328886  delta_E= -702  |g|= 0.585  |ddm|=  836
  HOMO = -0.236688626949668  LUMO = -0.149210306217129
cycle= 2 E= -0.166381149495714  delta_E= -0.907  |g|= 0.165  |ddm|=  836
  HOMO = -0.229732912204457  LUMO = -0.133913320544895
cycle= 3 E= -0.174823880766266  delta_E= -0.00844  |g|= 0.153  |ddm|= 0.198
  HOMO = -0.22618723497895  LUMO = -0.122680186864529
cycle= 4 E= -0.185911112344096  delta_E= -0.0111  |g|= 0.137  |ddm|= 0.284
  HOMO = -0.214519259339234  LUMO = -0.0986352417799118
cycle= 5 E= -0.203890005049614  delta_E= -0.018  |g|= 0.11  |ddm|= 0.594
  HOMO = -0.214069038106361  LUMO = -0.0842302934705603
cycle= 6 E= -0.213268898658577  delta_E= -0.00938  |g|= 0.0905  |ddm|= 0.432
  HOMO = -0.190929874401075  LUMO = -0.0236314569885834
cycle= 7 E= -0.229909345968174  delta_E= -0.0166  |g|= 0.0313  |ddm|= 1.67
  HOMO = -0.192629164694698  LUMO = -0.000654678353916344
cycle= 8 E= -0.232094721738326  delta_E= -0.00219  |g|= 0.00525  |ddm|= 0.445
  HOMO = -0.1

In [19]:
alchemical_potentials

[[0.46489912573415476,
  0.6752561816462017,
  0.5498145824722707,
  0.6996510402289364,
  0.622724210717711,
  0.4853744748005261,
  0.35351764384412754,
  0.36053399749821086,
  0.3863027335607996,
  0.3862965523847989,
  0.5007048644328371,
  0.5006793398776028]]

In [ ]:
e_homo

In [ ]:
e_lumo

In [ ]:
base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
save_obj(alchemical_potentials, base+'alch_pots')
save_obj(e_homo, base+'e_homo')
save_obj(e_lumo, base+'e_lumo')

### Plot results

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import qml
from ase.units import Bohr

import sys
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import alchemy_tools2 as at
import utils_qm as ut

In [ ]:
# load data
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')

base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
alchs = ut.load_obj(base+'alch_pots')
alch_pots = np.array(alchs)
alch_pots = np.concatenate((np.array([np.zeros(12)]), alch_pots))
homos = ut.load_obj(base+'e_homo')
homos.insert(0,0.0)
homos = np.array(homos)

lam_vals = np.array([0, 22, 30, 38, 44, 52])/52

In [ ]:
# # for guido
# np.savetxt('/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/alch_pots.txt', alch_pots)
# np.savetxt('/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/e_homos.txt', homos)
# np.savetxt('/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/lam_vals.txt', lam_vals)

### Calculation of atomic energies

integrate $\mu_I$ over lambda

In [ ]:
alch_pots_integrated = []
for i in range(alch_pots.shape[1]):
    alch_pots_integrated.append(np.trapz(alch_pots[:,i], x=lam_vals, axis=0))

In [ ]:
alch_pots_integrated

multiply by nuclear charge

In [ ]:
atomic_energies_without_rep = -(alch_pots_integrated*com.nuclear_charges)

In [ ]:
atomic_energies_without_rep.sum()

subtract nuclear repulsion

In [ ]:
nuc_rep_atomic = []
for i in lam_vals:
    nucs = i*com.nuclear_charges
    nuc_rep_atomic.append(at.nuclear_repulsion(nucs, com.coordinates/Bohr))

nuc_rep_atomic[-1]

In [ ]:
com.nuclear_charges

In [ ]:
nuc_rep_atomic[-1].sum() + atomic_energies_without_rep.sum()

In [ ]:
52*np.trapz(homos, x=lam_vals, axis=0)

In [ ]:
homos

In [ ]:
nuc_rep_atomic[-1].sum() + atomic_energies_without_rep.sum() + np.trapz(homos, x=lam_vals, axis=0)

In [ ]:
-360.6847443060918 + 358.7

In [ ]:
com.nuclear_charges

In [ ]:
for alch, nuc_charge in alch_pots:

calculation of $\sum_I \mu_I(\lambda)$, $\sum_I E_I(\lambda)$ and $\lambda$

In [ ]:
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
total_elec = com.nuclear_charges.sum()

alch_pots_sum = []
atomic_energies_sum = []
for i in alch_pots:
    atomic_energies_sum.append((i*com.nuclear_charges).sum())
    print((i*com.nuclear_charges).sum())
    alch_pots_sum.append(i.sum())

alch_pots_sum.insert(0,0.0)  
atomic_energies_sum.insert(0,0.0) 


calculation of atomic nuclear repulsion at every lambda $E_{I, \rm{NN}}(\lambda)$ and atomic energy with repulsion

In [ ]:
nuc_rep_atomic = []
for i in lam_vals:
    nucs = i*com.nuclear_charges
    nuc_rep_atomic.append(at.nuclear_repulsion(nucs, com.coordinates/Bohr))

In [ ]:
atomic_en_with_nuc_rep = []
for alch, nuc_rep, nuc_charge in zip(alch_pots, nuc_rep_atomic):
    atomic_minus_nu.append(alch-nuc)


In [ ]:
sum_at = []
for i in atomic_minus_nu:
    sum_at.append(i.sum())

In [ ]:
np.array(sum_at).sum()

In [ ]:
np.trapz(sum_at, x=lam_vals, axis=0)

In [ ]:
fig, ax = plt.subplots(1,1)
plt.rcParams['font.size'] = 18
ax.plot(lam_vals, alch_pots_sum, '-o', label='$\sum_I \mu_I$')
ax.plot(lam_vals, atomic_energies_sum, '-o', label='$\sum_I Z_I \mu_I$')

ax.legend()
ax.set_ylabel('$\sum_I \mathcal{P}_I$')
ax.set_xlabel('$\lambda$')
# plt.savefig('/home/misa/projects/Atomic-Energies/figures/different_resevoir/com_001212_alch_pot_atomic_en.png', bbox_inches='tight')

- Why is integral over not equal to total energy at lambda=1? Is the nuc-nuc repulsion included?
What is the expression for $E(\lambda)$, what is the integral over the whole path?
- When/How to add e homo? For every lambda or only lambda = 1?
- Are there still some bugs in the code?
- Are the EPN's equal to the alchemical potential or the atomic energy?
- Calculate some lower values for lambda?

Plots vs $\lambda$:
- $E(\lambda)$ 
- $\sum_I \mu_I(\lambda)$
- $\sum_I E_I(\lambda)$
- $\int_0^\lambda E_I(\lambda') d\lambda'$

In [ ]:
# com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
# total_elec = com.nuclear_charges.sum()
# total_num_ve = 38
# num_ves = [8, 16, 24, 30, 38]
# alchemical_potentials = []
# e_homo = []
# e_lumo = []

# for num_ve in num_ves:
#     print(f'Calculating {num_ve}')
#     lam_val = num_ve/total_num_ve
#     # core_elec = com.nuclear_charges.sum() - total_num_ve

#     mol = pyscf.gto.Mole()
#     for ch, coords in zip(com.nuclear_charges, com.coordinates):
#         mol.atom.append([ch, coords])

#     mol.basis = 'def2-tzvp'
#     mol.charge = total_num_ve - num_ve
#     mol.verbose = 0
#     mol.build()

#     method = "HF"
#     if method not in ["CCSD", "HF"]:
#         raise NotImplementedError("Method %s not supported." % method)

#     deltaZ = -com.nuclear_charges*(1-lam_val)
#     includeonly = np.arange(len(mol.atom_coords()))


#     def add_qmmm(calc, mol, deltaZ):
#         mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()[includeonly]/ angstrom, deltaZ)

#         def energy_nuc(self):
#             q = mol.atom_charges().astype(np.float)
#             q[includeonly] += deltaZ
#             return mol.energy_nuc(q)

#         mf.energy_nuc = energy_nuc.__get__(mf, mf.__class__)

#         return mf

#     if method == "HF":
#         calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
#         hfe = calc.kernel(verbose=0)
#         dm1_ao = calc.make_rdm1()
#         total_energy = calc.e_tot
#     if method == "CCSD":
#         calc = add_qmmm(pyscf.scf.RHF(mol), mol, deltaZ)
#         hfe = calc.kernel(verbose=0)
#         mycc = pyscf.cc.CCSD(calc).run()
#         dm1 = mycc.make_rdm1()
#         dm1_ao = np.einsum("pi,ij,qj->pq", calc.mo_coeff, dm1, calc.mo_coeff.conj())
#         total_energy = mycc.e_tot

#     # GRIDLESS, as things should be ############################
#     # Total energy of SCF run

#     print("TOTAL_ENERGY", total_energy)

#     # Electronic EPN from electron density
#     alch_pot_tmp = []
#     for site in includeonly:
#         mol.set_rinv_orig_(mol.atom_coords()[site])
# #         print("ELECTRONIC_EPN", site, np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
#         alch_pot_tmp.append(np.matmul(dm1_ao, mol.intor("int1e_rinv")).trace())
#     alchemical_potentials.append(alch_pot_tmp)
    
#     # energy of HOMO and LUMO
#     nocc = int((total_elec - mol.charge)/2)
#     e_homo.append(calc.mo_energy[nocc-1])
#     e_lumo.append(calc.mo_energy[nocc])